### 3.5. Preparación de los datos para redes neuronales

Para implementar los modelos **LSTM** y **GRU**, fue necesario reestructurar los datos históricos en forma de secuencias temporales, adecuadas para el procesamiento en redes neuronales recurrentes.  
Este paso resulta esencial, ya que estos modelos aprenden patrones en el tiempo a partir de ventanas consecutivas de datos, en lugar de observaciones independientes.

**• Creación de secuencias:**  
Se generaron ventanas deslizantes (*sliding windows*) con un tamaño de **10 días consecutivos** (`window_size = 10`).  
Cada ventana se utilizó como entrada para predecir el **precio de cierre (Close)** del día siguiente (*horizonte t+1*).  
Este enfoque, ampliamente utilizado en modelado de series temporales, permite capturar patrones locales y relaciones de corto plazo entre los indicadores y el comportamiento del precio.

**• División en entrenamiento y prueba:**  
El conjunto de datos se dividió respetando el orden temporal, asignando el **80 % de las observaciones para entrenamiento** y el **20 % restante para prueba**.  
De este modo se evita la fuga de información (*data leakage*) y se simulan condiciones reales de predicción.  
Cada subconjunto se secuenció de forma independiente, asegurando que las ventanas de prueba no utilicen información del bloque de entrenamiento.

**• Formato de los datos:**  
Las secuencias generadas para los modelos recurrentes tienen dimensiones de:


$X \in \mathbb{R}^{(n_{\text{samples}},\; window\_size,\; n_{\text{features}})}$


donde:
-  $n_{\text{samples}}$ : número de secuencias creadas.  
- $ window\_size = 10$ : tamaño de la ventana temporal (días consecutivos).  
- $n_{\text{features}}$: número de variables predictoras (precios e indicadores técnicos).

Las etiquetas correspondientes $(y_{train}, y_{test})$ son vectores unidimensionales que contienen el **precio de cierre del día siguiente** a cada secuencia.  
Este formato garantiza la compatibilidad con arquitecturas LSTM y GRU, las cuales esperan tensores tridimensionales en su capa de entrada.


In [1]:
# ============================================
# 0) Imports y rutas
# ============================================
import numpy as np
import pandas as pd
from pathlib import Path

IN_DIR  = Path("../data/processed/ready_for_modeling")
OUT_DIR = Path("../data/processed/nn_ready")
OUT_DIR.mkdir(parents=True, exist_ok=True)

TICKERS = ["BBVA", "SAN"]
WINDOW_SIZE = 10          # tamaño de ventana
HORIZON = 1               # predecimos t+1
TEST_SIZE = 0.20          # 80/20 respetando tiempo
TARGET_COL = "Close"      # predicción sobre Close (escalado 0–1)

# ============================================
# 1) Utilidades
# ============================================
def split_index(n, test_ratio=0.2):
    """Índice de corte para división temporal."""
    return int(n * (1 - test_ratio))

def make_sequences(df, window=10, horizon=1, target_col="Close"):
    """
    Genera secuencias deslizantes (X) y objetivo (y) a partir de un DataFrame.
    X tiene forma (n_samples, window, n_features)
    y tiene forma (n_samples,)
    """
    values = df.values.astype(np.float32)
    y_idx = df.columns.get_loc(target_col)

    X_list, y_list = [], []
    # última posición que permite horizonte completo
    last_start = len(df) - window - horizon + 1
    for start in range(last_start):
        end = start + window
        X_list.append(values[start:end, :])
        y_list.append(values[end + horizon - 1, y_idx])

    X = np.stack(X_list, axis=0)
    y = np.array(y_list, dtype=np.float32)
    return X, y

def timewise_train_test_sequences(df, window=10, horizon=1, target_col="Close", test_ratio=0.2):
    """
    Divide por tiempo ANTES de secuenciar para evitar fuga:
    - Todas las ventanas del train quedan dentro del bloque de train
    - Todas las ventanas del test quedan dentro del bloque de test
    """
    # índice de corte para el bloque base
    cut = split_index(len(df), test_ratio)
    df_train_block = df.iloc[:cut].copy()
    df_test_block  = df.iloc[cut:].copy()

    # Para que una ventana de test sea válida, necesita 'window' pasos antes dentro del propio bloque
    # y 'horizon' pasos después. Por eso desplazamos el inicio en test.
    # Train:
    X_train, y_train = make_sequences(df_train_block, window=window, horizon=horizon, target_col=target_col)

    # Test: necesita como mínimo window + horizon - 1 puntos dentro de su bloque
    if len(df_test_block) >= window + horizon:
        X_test, y_test = make_sequences(df_test_block, window=window, horizon=horizon, target_col=target_col)
    else:
        X_test = np.empty((0, window, df.shape[1]), dtype=np.float32)
        y_test = np.empty((0,), dtype=np.float32)

    return X_train, X_test, y_train, y_test

# ============================================
# 2) Proceso por ticker
# ============================================
for t in TICKERS:
    df = pd.read_csv(IN_DIR / f"{t}_final_ready.csv", parse_dates=["Date"], index_col="Date").sort_index()

    # Asegurar que target existe
    assert TARGET_COL in df.columns, f"{t}: no existe la columna {TARGET_COL}"

    # Generar secuencias preservando orden temporal (sin fuga)
    X_train, X_test, y_train, y_test = timewise_train_test_sequences(
        df, window=WINDOW_SIZE, horizon=HORIZON, target_col=TARGET_COL, test_ratio=TEST_SIZE
    )

    # Guardar tensores para Keras/PyTorch
    np.savez_compressed(
        OUT_DIR / f"{t}_ws{WINDOW_SIZE}_h{HORIZON}_nn_ready.npz",
        X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test,
        feature_names=np.array(df.columns)
    )

    # Resumen
    n_features = X_train.shape[2] if X_train.size else df.shape[1]
    print(f"✅ {t}: guardado -> {OUT_DIR}/{t}_ws{WINDOW_SIZE}_h{HORIZON}_nn_ready.npz")
    print(f"   Shapes → X_train: {X_train.shape}, y_train: {y_train.shape}, "
          f"X_test: {X_test.shape}, y_test: {y_test.shape} | n_features={n_features}\n")


✅ BBVA: guardado -> ..\data\processed\nn_ready/BBVA_ws10_h1_nn_ready.npz
   Shapes → X_train: (5282, 10, 16), y_train: (5282,), X_test: (1313, 10, 16), y_test: (1313,) | n_features=16

✅ SAN: guardado -> ..\data\processed\nn_ready/SAN_ws10_h1_nn_ready.npz
   Shapes → X_train: (5282, 10, 16), y_train: (5282,), X_test: (1313, 10, 16), y_test: (1313,) | n_features=16



**Resultado del preprocesamiento para redes neuronales (punto 3.5):**

El proceso de preparación de datos se completó correctamente para ambos activos (BBVA y Santander).  
Se generaron secuencias deslizantes de 10 días consecutivos para predecir el precio de cierre del día siguiente (horizonte t+1).  
La división temporal se realizó en un 80 % para entrenamiento y un 20 % para prueba, respetando el orden cronológico y evitando fugas de información.

Los conjuntos de datos finales presentan las siguientes dimensiones:

| Banco | X_train | y_train | X_test | y_test | n_features |
|--------|----------|----------|---------|---------|-------------|
| **BBVA** | (5282, 10, 16) | (5282,) | (1313, 10, 16) | (1313,) | 16 |
| **SAN**  | (5282, 10, 16) | (5282,) | (1313, 10, 16) | (1313,) | 16 |

Cada secuencia contiene 10 pasos temporales y 16 variables predictoras (precios e indicadores técnicos).  
Los archivos fueron almacenados en `../data/processed/nn_ready/`, listos para ser utilizados en la implementación de las redes **LSTM** y **GRU** del apartado siguiente.
